In [1]:
import os
from os.path import join, pardir
from collections import Counter
from copy import deepcopy
import numpy as np
from deap import base, creator, algorithms, tools
from dssg_challenge import compute_cost, check_keyboard

RNG_SEED = 0
DATA_DSSG = join(pardir, 'data', 'raw')

rng = np.random.RandomState(RNG_SEED)

In [2]:
os.listdir(DATA_DSSG)

['pt-corpus.txt', '.gitkeep', 'pt-keys.txt', 'en-keys.txt', 'en-corpus.txt']

In [3]:
# get keys
with open(join(DATA_DSSG, 'en-keys.txt'), 'r') as file:
    keys = file.read()

# get corpus example
with open(join(DATA_DSSG, 'en-corpus.txt'), 'r') as file:
    corpus = file.read()

keys = ''.join(keys.split('\n'))
corpus = ''.join(corpus.split(keys)).split('\n')[0]

Some keys are used to signal special characters. Namely,

- The ENTER key is represented as 0.
- The shift key for capitalization is represented as ^.
- The backspace key is represented as <.
- All the remaining characters not found in the valid keys are encoded as #.
- Empty keys will contain the character _.


In [4]:
len(keys), keys

(34, 'ABCDEFGHIJKLMNOPQ RSTUVWXYZ0.#,^?<')

## The most basic approaches

In [5]:
Counter(corpus).most_common()[:10]

[(' ', 83),
 ('T', 45),
 ('I', 45),
 ('O', 39),
 ('E', 35),
 ('A', 31),
 ('S', 27),
 ('N', 21),
 ('L', 20),
 ('H', 19)]

In [6]:
baseline = ''.join([i[0] for i in Counter(corpus).most_common()])
baseline = baseline + ''.join([i for i in keys if i not in baseline]) + '  T'
baseline

' TIOEASNLHRM<DVW#PYKFGUC0BZ.^J,QX?  T'

In [7]:
shuffled = list(baseline)
rng.shuffle(shuffled)

anthony = 'EINOA TCGVDURL<^SWH_Z__XJQFPBMY,#.0K?'

check_keyboard(baseline, keys)
check_keyboard(keys+'  T', keys)
check_keyboard(shuffled, keys)
check_keyboard(''.join([i if i!='_' else ' ' for i in anthony]), keys)

print('Shuffled cost:\t\t\t', compute_cost(''.join(shuffled), corpus))
print('Original keys cost:\t\t', compute_cost(keys+' ', corpus))
print('Baseline cost:\t\t\t', compute_cost(baseline, corpus))
print('Anthony Carbajal\'s solution:\t', compute_cost(''.join([i for i in anthony if i!='_']), corpus))

Shuffled cost:			 2421.8189712388935
Original keys cost:		 2384.127991417558
Baseline cost:			 1792.1951037269362
Anthony Carbajal's solution:	 1737.8888937148206


## First attempt with GA

In [8]:
keys_list = list(keys)

def evaluate(individual):
    """
    Computes the cost for each individual.
    """
    try:
        check_keyboard(individual, keys)
        return [compute_cost(''.join(list(individual)), corpus)]
    except AssertionError:
        return [np.inf]

def mutFlip(ind1, ind2):
    """Execute a two points crossover with copy on the input individuals. The
    copy is required because the slicing in numpy returns a view of the data,
    which leads to a self overwritting in the swap operation.
    """

    ind = ind1.copy()
    for x, value in np.ndenumerate(ind):
        if np.random.random() < .05:
            ind[x] = np.random.choice(keys_list)
    try:
        check_keyboard(ind, keys)
        return ind, ind2
    except AssertionError:
        return mutFlip(individual, ind2)
    
    return ind, ind2


In [19]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', np.ndarray, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Tool to randomly initialize an individual
toolbox.register('attribute',
        np.random.permutation, np.array(list(baseline))
)

toolbox.register('individual',
    tools.initIterate,
    creator.Individual,
    toolbox.attribute
)

toolbox.register('population',
    tools.initRepeat,
    list,
    toolbox.individual
)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    np.random.seed(64)

    pop = toolbox.population(n=20)

    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1, similar=np.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0, mutpb=0.6, ngen=1000, stats=stats,
                        halloffame=hof)

    return pop, stats, hof


pop, stats, hof = main()


gen	nevals	avg    	std    	min    	max   
0  	20    	2443.51	127.229	2260.53	2715.8
1  	12    	2362.51	67.4254	2265.36	2498.43
2  	7     	2319.66	47.0608	2232.48	2410.08
3  	10    	2275.28	33.669 	2203.86	2341.36
4  	13    	2238   	39.0845	2167.37	2296.41
5  	13    	2206.16	27.8452	2158.39	2265.36
6  	15    	2215.16	55.1014	2158.39	2413.04
7  	9     	2194.22	41.396 	2138.07	2304.29
8  	12    	2197.75	56.0483	2138.07	2370.66
9  	10    	2183.55	57.4832	2113.84	2390.77
10 	11    	2159.85	48.5225	2041.63	2281.75
11 	15    	2162.64	92.7278	2038.85	2444.91
12 	8     	2110.16	56.9187	2023.31	2201.81
13 	12    	2106.56	71.7515	2023.31	2296.21
14 	8     	2080.28	75.6044	2014.11	2271.95
15 	11    	2063.89	55.1827	2006.76	2184.43
16 	9     	2044.98	41.9888	1986.17	2157.98
17 	9     	2056.63	70.9669	1986.17	2255.38
18 	12    	2039.64	61.0464	1945.92	2237.38
19 	11    	2017.1 	47.5858	1945.92	2179.59
20 	13    	2025.83	74.2228	1945.92	2195.64
21 	13    	1991.96	45.5372	1927.39	2086.44
22 	14    	19

190	13    	1773.61	72.9345	1715.98	1965.53
191	11    	1795.31	95.7628	1715.98	1995.38
192	12    	1770.53	69.6894	1715.98	1945.86
193	14    	1797.37	106.703	1715.98	2054.94
194	13    	1764.72	64.3281	1715.98	1901.13
195	13    	1771.07	75.8325	1715.98	1950.38
196	13    	1767.18	53.2732	1715.98	1865.14
197	11    	1771.46	81.3999	1715.98	2037.64
198	9     	1757.05	75.9945	1710.99	1972.34
199	10    	1748.13	52.4068	1710.82	1859.64
200	12    	1778.41	100.758	1710.82	2066.62
201	11    	1789.41	92.2738	1709.08	2016.1 
202	11    	1755.67	72.4321	1709.08	1923.47
203	12    	1787.87	98.8758	1709.08	2023.18
204	11    	1774.08	95.1107	1709.08	1981.9 
205	13    	1782.62	107.171	1705   	2111.4 
206	13    	1807.44	106.156	1705   	2047.9 
207	16    	1833.01	99.7225	1705   	2008.66
208	7     	1753.49	66.6417	1681.42	1919.27
209	11    	1756.09	64.8597	1704.86	1898.45
210	9     	1762.72	80.2282	1704.86	2007.92
211	9     	1755.03	88.3302	1704.86	2040.92
212	16    	1803.15	123.693	1699.5 	2200.35
213	11    	

381	9     	1751.84	100.797	1673.9 	2031.38
382	11    	1759.24	89.0381	1673.9 	2051.73
383	13    	1761.24	82.1838	1673.9 	1940   
384	13    	1737.89	89.4323	1667.5 	1986.75
385	10    	1727.88	81.4432	1667.5 	1960.71
386	15    	1743.72	47.3091	1673.9 	1805.4 
387	10    	1754.59	61.1515	1673.9 	1883.06
388	10    	1740.15	79.5828	1673.9 	1996.06
389	12    	1724.28	69.4013	1673.9 	1890.25
390	11    	1729.48	94.8104	1673.9 	2042.08
391	14    	1757.7 	99.3083	1673.9 	1984.56
392	14    	1767.82	119.493	1673.81	2104.99
393	13    	1743.81	74.5146	1672.6 	1909.32
394	10    	1734.53	77.3748	1672.6 	1927.9 
395	12    	1725.8 	86.0581	1672.6 	2026.77
396	11    	1701.19	47.0912	1672.6 	1850.12
397	14    	1739.68	79.6631	1672.6 	1969.77
398	12    	1776.93	115.137	1645.99	2084.68
399	11    	1769.78	94.3612	1645.99	1929.49
400	10    	1719.2 	69.5086	1645.99	1845.48
401	12    	1734.27	96.2794	1645.99	1942.67
402	9     	1719.94	85.6995	1645.99	1913.46
403	15    	1736.69	79.3067	1645.99	1878.11
404	11    	

573	17    	1743.83	122.787	1599.91	1967.82
574	12    	1709.79	93.3694	1602.55	1966.81
575	12    	1714.26	115.929	1602.55	1943.53
576	12    	1662.03	63.6358	1602.55	1819.62
577	13    	1705.07	98.1379	1602.55	1911   
578	12    	1681.9 	96.2908	1601.9 	1934.35
579	12    	1670.34	108.224	1601.9 	2048.24
580	13    	1669.7 	107.372	1601.9 	1898.78
581	13    	1676.21	86.5163	1601.9 	1884.14
582	9     	1684.62	155.292	1601.9 	2180.84
583	8     	1663.36	104.477	1601.9 	1982.62
584	11    	1654.2 	94.6923	1601.9 	1940.11
585	13    	1684.85	103.221	1601.9 	1969.31
586	11    	1668.83	80.382 	1601.9 	1877.75
587	12    	1668.97	118.364	1601.9 	2067.99
588	13    	1692.38	111.456	1601.9 	1983.02
589	11    	1676.41	102.603	1601.9 	1908.22
590	13    	1706.28	109.408	1601.9 	1933.61
591	14    	1710.35	125.155	1601.9 	1974.44
592	15    	1694.52	79.9287	1601.9 	1841.84
593	15    	1711.16	112.594	1601.9 	1947.3 
594	16    	1713.34	130.429	1601.9 	2024.79
595	15    	1716.22	113.96 	1601.9 	1992.27
596	13    	

764	12    	1686.12	103.737	1606.7 	1920.72
765	10    	1726.3 	146.852	1606.7 	2080.97
766	11    	1701.92	123.774	1606.7 	2036.86
767	12    	1672.58	90.6822	1606.7 	1914.05
768	12    	1666.7 	79.2583	1606.7 	1846.54
769	12    	1701.22	129.225	1606.7 	2048.8 
770	16    	1748.32	128.113	1606.7 	2092.3 
771	16    	1724.16	94.072 	1606.7 	1932.93
772	10    	1718.65	114.674	1606.7 	2022.59
773	15    	1735.45	122.601	1606.7 	1997.33
774	10    	1689.55	78.1433	1606.7 	1915.07
775	12    	1665.4 	78.4737	1606.7 	1867.71
776	12    	1699.11	111.317	1606.7 	2003.63
777	11    	1695.58	95.8232	1606.7 	1878.01
778	16    	1691.68	77.0135	1606.7 	1861.71
779	13    	1683.93	100.177	1606.7 	1939.94
780	15    	1679.02	78.6314	1606.7 	1850.18
781	12    	1710.94	82.3969	1606.7 	1840.77
782	13    	1728.36	139.419	1606.7 	2144.07
783	12    	1717.21	106.243	1617.32	1967.89
784	13    	1742.87	102.701	1617.32	2036.97
785	11    	1736.36	115.888	1617.32	2053.99
786	13    	1722.06	108.904	1617.32	1969.91
787	10    	

955	12    	1735.56	83.9693	1651.01	1953.96
956	14    	1724.24	72.0179	1651.01	1961.08
957	7     	1709.31	72.1512	1651.01	1886.32
958	11    	1694.08	48.2325	1651.01	1814.64
959	12    	1721.71	87.3441	1651.01	1908.98
960	12    	1711.43	61.8313	1651.01	1831.91
961	13    	1727.02	101.22 	1651.01	2015.84
962	11    	1701.03	73.995 	1648.73	1920.53
963	10    	1704.49	92.9605	1651.01	2029.87
964	13    	1720.27	89.9157	1651.01	1965.69
965	9     	1704.58	69.2208	1651.01	1863.95
966	11    	1731.86	103.592	1651.01	2047.66
967	13    	1727.93	78.3342	1651.01	1911.01
968	13    	1731.28	97.2869	1651.01	2031.23
969	13    	1736.22	76.0796	1651.01	1874.93
970	13    	1724.68	74.1963	1651.01	1894.22
971	14    	1772.4 	108.953	1651.01	2069.1 
972	11    	1728.97	93.6183	1651.01	1944.8 
973	11    	1700.55	66.9109	1651.01	1846.8 
974	15    	1721.97	57.8411	1651.01	1838.13
975	14    	1724.6 	73.8768	1651.01	1893.27
976	13    	1751.13	116.228	1651.01	2041.6 
977	13    	1731.51	86.436 	1640.76	1899.67
978	14    	

In [20]:
''.join(list(hof)[0])

'OSNA ETM GWYPRLV HI#.0^<J?BKC,FTUDQZX'

In [21]:
check_keyboard('OSNA ETM GWYPRLV HI#.0^<J?BKC,FTUDQZX', keys)
compute_cost('OSNA ETM GWYPRLV HI#.0^<J?BKC,FTUDQZX', corpus)

1599.9106808456932

## Hall of fame solutions

    ' ONYTIAIMZGBCHEDRSL,P#.^0TQX VK?W<JFU' - 1673.418
    ' REASTO<DGWVPMILNYHTJC ^0. #?X,QUBZFK' - 1637.709
    ' RDSTOECP#<WINALGYKX , ^0.ZMFHUJVBT?Q' - 1582.775
    'T OISLADERNMGW #UYHTVKCFPX<, ?ZJ.0^BQ' - 1597.119
    'OSNA ETM GWYPRLV HI#.0^<J?BKC,FTUDQZX' - 1599.910